In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from collections import Counter, defaultdict

# 1. 웹 드라이버 설정
driver = webdriver.Chrome()  # 크롬 드라이버 사용
driver.get('https://www.naver.com')

# 2. 사용자 입력 받기
search_query = input("요리 이름을 입력하세요: ")

# 3. 네이버에서 레시피 검색
search_box = driver.find_element(By.NAME, 'query')  # 네이버 검색창
search_box.send_keys(f"{search_query} 레시피")
search_box.send_keys(Keys.RETURN)

# 4. 검색 결과에서 레시피 링크 수집
wait = WebDriverWait(driver, 10)
recipe_links = []

try:
    results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="https://blog.naver.com/"]')))
    recipe_links = [result.get_attribute('href') for result in results][:5]  # 상위 5개 레시피 링크 수집
except Exception as e:
    print(f"검색 결과를 가져오는 중 오류 발생: {e}")
    driver.quit()

# 5. 각 레시피에서 재료 크롤링
ingredients_list = []
recipe_ingredients = []  # 각 레시피별 재료 리스트

for link in recipe_links:
    driver.get(link)
    try:
        # 여기에 각 블로그에서 재료 목록을 찾는 코드를 넣습니다.
        # 블로그 구조에 따라 적절한 selector를 사용해야 합니다.
        ingredients = driver.find_elements(By.CSS_SELECTOR, '.se-component.se-text.se-l-default')  # 재료 리스트가 있는 요소를 선택
        recipe_ingredient = []
        for ingredient in ingredients:
            ingredient_text = ingredient.text.strip()
            ingredients_list.append(ingredient_text)
            recipe_ingredient.append(ingredient_text)
        recipe_ingredients.append(recipe_ingredient)
    except Exception as e:
        print(f"레시피에서 재료를 가져오는 중 오류 발생: {e}")

# 6. 재료 집계 및 대체 식품 분석
ingredients_counter = Counter()
substitutions = defaultdict(list)

for ingredients in recipe_ingredients:
    for ingredient in ingredients:
        ingredients_counter[ingredient] += 1

    # 각 레시피에서 빠진 주요 재료를 대신하는 재료 찾기
    for common_ingredient in ingredients_counter.keys():
        if common_ingredient not in ingredients:
            substitutions[common_ingredient].extend(ingredients)

# 7. 대체 식품 분석 결과 집계
for key in substitutions:
    substitutions[key] = Counter(substitutions[key]).most_common(1)

# 8. 결과 출력
print("\n가장 많이 사용된 재료들:")
for ingredient, count in ingredients_counter.most_common():
    substitute_text = ""
    if substitutions[ingredient]:
        substitute, sub_count = substitutions[ingredient][0]
        substitute_text = f"(대체 식품: {substitute})"
    print(f"{ingredient}, {count}명 {substitute_text}")

# 9. 브라우저 종료
driver.quit()



가장 많이 사용된 재료들:
